>>> <hr> 모델 실습 <hr>

- SentenceClassifier 클래스는 임베딩 층을 구성할 떄 사용하는 단어 사전 크기(n_vocab)와 순환 신경망 클래스와 장단기 메모리 클래스에서 사용하는 매개변수를 입력으로 전달받는다.
- 모델 종류(model_type) 매개변수로 순환 신경망을 사용할지, 장단기 메모리를 사용할지 설정한다.

- 초기화 메서드에서는 SentenceClassifier 클래스에 입력된 함수의 매개변수에 따라 모델 구조를 미세 조정한다.
- 분류기 계층은 모델을 양방향으로 구성한다면 전달되는 입력 채널 수가 달라지므로 분류기 계층을 현재 모델 구조에 맞게 변경한다.

- 순방향 메서드에서는 입력받은 정수 인코딩을 임베딩 계층에 통과시켜 임베딩 값을 얻는다. 그리고 얻은 임베딩 값을 모델에 입ㅇ력하여 출력값을 얻는다.
- 출력값(output)의 마지막 시점만 활용할 예정이므로 [:, -1, :]으로 마지막 시점의 결괏값만 분리해 분류기 계층에 전달한다.

In [1]:
# 문장 분류 모델
from torch import nn

class SentenceClassifier(nn.Module):
    def __init__(
            self,
            n_vocab,
            hidden_dim,
            embedding_dim,
            n_layers,
            dropout=0.5,
            bidirectional=True,
            model_type="lstm"
    ):
        super().__init__()

        self.embedding=nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )
        if model_type =="rnn":
            self.model=nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type =="lstm":
            self.model=nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        if bidirectional:
            self.classifier=nn.Linear(hidden_dim * 2,1)
        else:
            self.classifier=nn.Linear(hidden_dim, 1)
        self.dropout=nn.Dropout(dropout)
    
    def forward(self, inputs):
        embeddings=self.embedding(inputs)
        output, _=self.model(embeddings)
        last_output=output[:, -1. :]
        last_output=self.dropout(last_output)
        logits=self.classifier(last_output)
        return logits

- SentenceClassifier 클래스를 선언했다면 모델 학습에 사용할 데이터세트를 불러온다. 데이터세트는 코포라 라이브러리의 네이버 영화 리뷰 감정 분석 데이터세트를 불러온다.

In [6]:
# 데이터세트 불러오기
import pandas as pd
from Korpora import Korpora
#%pip install tabulate

corpus=Korpora.load("nsmc")
corpus_df=pd.DataFrame(corpus.test)

train=corpus_df.sample(frac=0.9, random_state=42)
test=corpus_df.drop(train.index)

print(train.head(5).to_markdown())
print("Training Data Size :", len(train))
print("Testing Data Size :", len(test))

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons

In [7]:
# 데이터 토큰화 및 단어 사전 구축
from konlpy.tag import Okt
from collections import Counter

def build_vocab(corpus, n_vocab, special_tokens):
    counter=Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab=special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

tokenizer=Okt()
train_tokens=[tokenizer.morphs(review) for review in train.text]
test_tokens=[tokenizer.morphs(review) for review in test.text]

vocab=build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id={token: idx for idx, token in enumerate(vocab)}
id_to_token={idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '.', '이', '영화', '의', '..', '가', '에', '...']
5002
